## Generate Tilesets JSON

---

`earthengine upload image --manifest` does not allow for tifs across epsg zones. In this notebook we generate a file `tilesets.json` such that running

```bash
# https://github.com/wri/dl_exporter
dl_exporter manifest manifest_config.yaml tilesets.json gee_manifest.json
```

Will generate a manifest file `gee_manifest.json-<i>` for each (i-th) epsg zone.  Once you have those files you can create an `image_collection` and upload each epsg-specific image into the `image_collection`.

---

In [1]:
import re
import math
import pandas as pd
import mproc
from pprint import pprint
from descarteslabs.scenes import DLTile
import dl_exporter.utils as utils

---

### HELPER METHODS

In [2]:
H='^gs://dl-exports/wri_urban_india-2019/wri:ulu-india_'
T='_2019.tif$'

def extract_tile_id(path):
    path=re.sub(H,'',path)
    return re.sub(T,'',path)
    
    
def get_crs(tile_key):
    return DLTile.from_key(tile_key).crs


def get_crs_list(tile_keys):
    return [get_crs(k) for k in tile_keys]


def flatten(lst):
    l=[]
    for _l in lst:
        l+=_l
    return l

---

### GENERATE DATA CSV

In [3]:
"""
1. fetch file list 
2. insert into dataframe
3. add tile_key column
"""
!echo path > gcs_data.csv
!gsutil ls gs://dl-exports/wri_urban_india-2019/*.tif >> gcs_data.csv
df=pd.read_csv('gcs_data.csv')
df['tile_key']=df.path.apply(extract_tile_id)
r=df.sample().iloc[0]
print(df.shape[0],r.path,r.tile_key)

3449 gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:8:86_2019.tif 6144:0:5.0:42:8:86


---

### ADD CRS COLUMN

In [4]:
"""
1. create batched list of tile_keys
2. generate crs using multiprocessing
3. add crs column
"""
NB_BATCHES=16
TOTAL=df.shape[0]
BATCH_SIZE=int(math.ceil(TOTAL/NB_BATCHES))
BATCHED_KEYS=[df.tile_key[i*BATCH_SIZE:(i+1)*BATCH_SIZE].tolist() for i in range(NB_BATCHES)]

In [5]:
%time crs_lists=mproc.map_with_threadpool(get_crs_list,BATCHED_KEYS,max_processes=NB_BATCHES)

CPU times: user 13.5 s, sys: 2.84 s, total: 16.4 s
Wall time: 17.6 s


In [6]:
print('nb_batches:',len(crs_lists))
crs_list=flatten(crs_lists)
print('nb_crs_rows:',len(crs_list))
crs_list[:3]

nb_batches: 16
nb_crs_rows: 3449


['EPSG:32642', 'EPSG:32642', 'EPSG:32642']

In [7]:
df['crs']=crs_list
df.to_csv('lulc-india-2019.csv')
df.sample(10)

,path,tile_key,crs
1516,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:43:8:47,EPSG:32643
1418,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:43:7:39,EPSG:32643
1619,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:43:9:62,EPSG:32643
2507,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:44:2:71,EPSG:32644
2173,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:44:-7:37,EPSG:32644
1855,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:44:-2:67,EPSG:32644
1951,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:44:-4:45,EPSG:32644
1683,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:44:-10:39,EPSG:32644
3443,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:46:7:99,EPSG:32646
386,gs://dl-exports/wri_urban_india-2019/wri:ulu-i...,6144:0:5.0:43:-3:118,EPSG:32643


---

### CREATE URIS-JSON FOR MULTI-MANIFEST RUN

In [8]:
def to_tilesets(df):
    """ uris-json
    
    Creates a list of objects, each of which generates a seperate
    gee-mainfest for sources in a single epsg zone.
    """
    jsn={ c: []  for c in df.crs.unique().tolist() }
    for i,r in df.iterrows():
        jsn[r.crs].append(r.path)
    lst=[]
    for i,(c,u) in enumerate(jsn.items()):
        lst.append({
            'id': re.sub(":","",c),
            'crs': c,
            'sources': [{'uris': _u} for _u in u] })
    return lst

In [9]:
jsn=to_tilesets(df)

In [10]:
utils.save_json(jsn,'../tilesets.json')

In [11]:
pprint(utils.read_json('../tilesets.json'))

[{'crs': 'EPSG:32642',
  'id': 'EPSG32642',
  'sources': [{'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-1:79_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-1:80_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-1:82_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-1:83_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-1:84_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-1:85_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-2:83_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-2:84_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:42:-2:85_2019.

              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:66_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:67_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:68_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:69_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:70_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:71_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:72_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:73_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:-6:74_2019.tif'},
              {'uris': 'gs://dl-expor

              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:104_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:105_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:106_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:107_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:108_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:109_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:110_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:111_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:43:5:112_2019.tif'},
              {'uris': 'gs://dl-expor

              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:103_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:104_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:105_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:106_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:107_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:108_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:44_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:45_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:-3:46_2019.tif'},
              {'uris': 'gs://dl

              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:70_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:71_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:72_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:73_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:74_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:75_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:76_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:77_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:44:2:78_2019.tif'},
              {'uris': 'gs://dl-exports/wri_ur

              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:87_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:88_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:89_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:90_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:91_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:92_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:93_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:94_2019.tif'},
              {'uris': 'gs://dl-exports/wri_urban_india-2019/wri:ulu-india_6144:0:5.0:45:0:95_2019.tif'},
              {'uris': 'gs://dl-exports/wri_ur